# Etapa 01: Importação das bilbiotecas

In [ ]:
import bs4 as bs
import urllib.request
import re
import nltk
import numpy as np
import random
import string

In [ ]:
nltk.download('punkt')

In [ ]:
pip install -U spacy

In [ ]:
import spacy

In [ ]:
import spacy.cli
spacy.cli.download("pt_core_news_sm")

In [ ]:
nlp = spacy.load('pt_core_news_sm')

# Etapa 02: Seleção dos dados

In [ ]:
import urllib.request 
from bs4 import BeautifulSoup as bs

def indexarDados(urls):
    conteudo_completo = ""  
    
    for url in urls:
        try:
            link = urllib.request.urlopen(url) 
            conteudo = link.read() 
            conteudo_html = bs(conteudo, 'lxml') 
            
           
            paragrafos = conteudo_html.find_all('p')  
            conteudo = '' 
            
            for p in paragrafos:
                conteudo += p.text + "\n" 

            conteudo_completo += conteudo + "\n\n" 
        except Exception as e:
            print(f"Erro ao acessar a URL {url}: {e}")

    return conteudo_completo 


urls = [
    'https://pt.wikipedia.org/wiki/Cora%C3%A7%C3%A3o',
    'https://pt.wikipedia.org/wiki/Ventr%C3%ADculo',
    'https://pt.wikipedia.org/wiki/Doen%C3%A7a_cardiovascular'
]

conteudo_combined = indexarDados(urls)  

print(conteudo_combined)  


#  Etapa 03: Pré-Processamento da base

In [ ]:
conteudo = conteudo_combined.lower()
conteudo

In [ ]:
lista_sentencas = nltk.sent_tokenize(conteudo)

In [ ]:
lista_sentencas

In [ ]:
!python -m spacy download pt_core_news_sm 
import spacy

In [ ]:
pln = spacy.load("pt_core_news_sm") 
pln

In [ ]:
stop_words = spacy.lang.pt.stop_words.STOP_WORDS

In [ ]:
stop_words

In [ ]:
string.punctuation

In [ ]:
def preprocessamento(texto):
  texto = re.sub(r"https?://[A-Za-z0-9./]+", ' ', texto)
  documento = pln(texto)
  lista = []
  for token in documento:
    lista.append(token.lemma_)
  lista = [palavra for palavra in lista if palavra not in stop_words and palavra not in string.punctuation]
  lista = ' '.join([str(elemento) for elemento in lista if not elemento.isdigit()])
  return lista

In [ ]:
lista_sentencas_preprocessada = []
for i in range(len(lista_sentencas)):
  lista_sentencas_preprocessada.append(preprocessamento(lista_sentencas[i]))

In [ ]:
lista_sentencas_preprocessada

In [ ]:
for _ in range(5):
  i = random.randint(0, len(lista_sentencas) - 1)
  print(lista_sentencas[i])
  print(lista_sentencas_preprocessada[i])
  print('-----')

# Etapa 04: Calcular

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
frases_teste = lista_sentencas_preprocessada
frases_teste

In [ ]:
vetores_palavras = TfidfVectorizer()
palavras_vetorizadas = vetores_palavras.fit_transform(frases_teste)

In [ ]:
type(palavras_vetorizadas)

In [ ]:
palavras_vetorizadas

In [ ]:
!pip install --upgrade scikit-learn

print(vetores_palavras.get_feature_names_out())

In [ ]:
print(vetores_palavras.vocabulary_)

In [ ]:
print(vetores_palavras.idf_)

In [ ]:
palavras_vetorizadas.todense()

# Similaridade Cosseno

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similaridade = cosine_similarity(palavras_vetorizadas[0], palavras_vetorizadas)

In [ ]:
similaridade

In [ ]:
similaridade.argsort()

# Testar CHATBOT

In [ ]:
def responder(texto_usuario):
  resposta_chatbot = ''
  lista_sentencas_preprocessada.append(texto_usuario)

  tfidf = TfidfVectorizer()
  palavras_vetorizadas = tfidf.fit_transform(lista_sentencas_preprocessada)

  similaridade = cosine_similarity(palavras_vetorizadas[-1], palavras_vetorizadas)

  indice_sentenca = similaridade.argsort()[0][-2]
  vetor_similar = similaridade.flatten()
  vetor_similar.sort()
  vetor_encontrado = vetor_similar[-2]

  if (vetor_encontrado == 0):
    resposta_chatbot = resposta_chatbot + 'Desculpe, mas não entendi!'
    return resposta_chatbot
  else:
    resposta_chatbot = resposta_chatbot + lista_sentencas[indice_sentenca]
    return resposta_chatbot


In [ ]:
continuar = True
print('Olá, sou um chatbot e vou responder perguntas sobre inteligência artificial: ')
while (continuar == True):
  texto_usuario = input()
  texto_usuario = texto_usuario.lower()
  if (texto_usuario != 'sair'):
    if (responder_saudacao(texto_usuario) != None):
      print('Chatbot: ' + responder_saudacao(texto_usuario))
    else:
      print('Chatbot: ')
      print(responder(preprocessamento(texto_usuario)))
      lista_sentencas_preprocessada.remove(preprocessamento(texto_usuario))
  else:
    continuar = False
    print('Chatbot: Até breve!')